# Poke Platform DB Exploration

This notebook connects directly to RDS using Secrets Manager and loads results into pandas DataFrames.

Set `REGION` and `SECRET_ARN` as environment variables if you want to override defaults.


In [1]:
# %pip install pandas
%pip install boto3 pandas psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 4.2 MB 5.7 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
from pathlib import Path

# Ensure repo root is on sys.path for local imports
repo_root = Path().resolve().parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

import pandas as pd

from scripts.db_notebook_helpers import fetch_db_secret, read_sql

REGION = os.environ.get("REGION", "us-east-2")
SECRET_ARN = os.environ.get("SECRET_ARN", "arn:aws:secretsmanager:us-east-2:878817878019:secret:PokePlatformStackPostgresSe-uD7oKVjyUmYi-NFBVMe")


## Quick sanity checks


In [3]:
read_sql("SELECT COUNT(*) AS active_tracked FROM tracked_asset WHERE is_active=true;", region=REGION, secret_arn=SECRET_ARN)


/home/ec2-user/.local/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


OperationalError: connection to server at "pokeplatformstack-postgres9dc8bb04-bmdwvuufi4pj.cpiyqo622ydt.us-east-2.rds.amazonaws.com" (10.0.224.153), port 5432 failed: timeout expired


In [ ]:
read_sql("SELECT COUNT(*) AS card_metadata_count FROM card_metadata;", region=REGION, secret_arn=SECRET_ARN)


In [ ]:
read_sql("SELECT MAX(updated_ts) AS latest_updated_ts FROM card_metadata;", region=REGION, secret_arn=SECRET_ARN)


## Sample slice from card_metadata


In [4]:
df_cards = read_sql("SELECT asset_id, name, set_name, rarity, updated_ts FROM card_metadata ORDER BY updated_ts DESC LIMIT 50;", region=REGION, secret_arn=SECRET_ARN)
df_cards.head(10)


OperationalError: connection to server at "pokeplatformstack-postgres9dc8bb04-bmdwvuufi4pj.cpiyqo622ydt.us-east-2.rds.amazonaws.com" (10.0.224.153), port 5432 failed: timeout expired


## Your custom query


In [ ]:
custom_sql = """
SELECT *
FROM tracked_asset
WHERE is_active=true
LIMIT 50;
"""
df_custom = read_sql(custom_sql, region=REGION, secret_arn=SECRET_ARN)
df_custom
